Обработка датасета почты России

По предварительному анализу можно сказать что оба датасета, и трайн и тест сделаны из одного датасета, по id можно понять что разбиение было через строку.
  В самомо датасете имеються как категориальные так и количественные показатели, при этом есть еще и текстовый признак (name_mfi), который так же нуждается в обработке. дополнительно можно скачать справочник индексов, он находится на сайте почты россии и имеется в открытом доступе.

1. Установим (догрузим) необходимые библиотеки

In [ ]:
!pip download -r requires.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 KB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.0/770.0 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━

# Новый раздел

In [ ]:
!pip install googletrans==3.1.0a0
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 1.4 MB 49.9 MB/s 
     |████████████████████████████████| 65 kB 3.3 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16370 sha256=373e43f5727bfe543a1d8dbebd6d658d408af85ad90c02d7f1ad612d6f7bbe32
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 82 kB/s 


2. Подключим имеющиеся библиотеки

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier as CBclass
%matplotlib inline
import gc
import os
import spacy

import datetime
import re
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

import googletrans
import csv
from googletrans import Translator

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  


3. в связи с тем что работа производится в колабе, скачаем все датасеты в память, которые размещены на google диске

In [ ]:
! [ ! -s "./train_dataset_train.csv" ] && curl -L -o train_dataset_train.csv "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1qVRFpW_mq-A--xT26ds78APDjRquzrVB"
! [ ! -s "./PIndx20.csv" ] && curl -L -o PIndx20.csv "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1inuDJ_CRsWIUy_14It3iey2RlCqq3_6b"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  949M  100  949M    0     0   136M      0  0:00:06  0:00:06 --:--:--  149M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 7518k  100 7518k    0     0  5225k      0  0:00:01  0:00:01 --:--:-- 5225k


Подключим google диск для сохранности данных и последующей обработки, можно было бы работать и напрямую но мне кажется будет просадка в скорости, поэтому качаем

обьявим пременные для датасетов

In [ ]:
df_index = pd.read_csv("/content/PIndx20.csv", sep=";")
df_train= pd.read_csv("/content/train_dataset_train.csv")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


добавим в датасет, скачанный с почты России новый признак - время жизни сведений с момента последнего их обновления и по 8.11.2022

In [ ]:
df_index.info()
df_index['date'] = pd.to_datetime(df_index['ACTDATE'], format='%d.%m.%Y')
c = '08.11.2022'
df_index['st'] = pd.to_datetime(c, format='%d.%m.%Y')
df_index['LifeDate'] =  (df_index['st'] - df_index ['date']).dt.days
print(df_index['LifeDate'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58005 entries, 0 to 58004
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   INDEX     58005 non-null  int64  
 1   OPSNAME   58005 non-null  object 
 2   OPSTYPE   58005 non-null  object 
 3   OPSSUBM   57918 non-null  float64
 4   REGION    57446 non-null  object 
 5   AUTONOM   560 non-null    object 
 6   AREA      37612 non-null  object 
 7   CITY      51842 non-null  object 
 8   CITY_1    2102 non-null   object 
 9   ACTDATE   58005 non-null  object 
 10  INDEXOLD  0 non-null      float64
dtypes: float64(2), int64(1), object(8)
memory usage: 4.9+ MB
0        4309
1        3913
2        2000
3         965
4        1027
         ... 
58000     616
58001     616
58002     616
58003     616
58004     613
Name: LifeDate, Length: 58005, dtype: int64


Напишем функцию для обработки текств name MFI,
 она будет включать в себя :
 1. перевод регистра в нижний
 сгруппируем данные и выведем в отдельный датафрейм, чтобы работать не с 6 млн записями а с 202000.
 2. удалим все цифры(много моделей, серийных номеров.их можно опустить)
 2. приведение к единому языку
    1. считываем ячейку,
    2. узнаем что за язык,
    3. оставим вероятности для выяснения возможности ошибочного написания, при этом некоторые строки в связи с использованием слэнга трогать не будем
 3. попробуем сформировать категории

In [ ]:
 # поработаем с наименованиями отправлений
def obrabotka_text(df):
    df["name_mfi"] = df["name_mfi"].fillna('no_data')#заполняем пропущенные значения
    df.loc[df["name_mfi"] == '0', "name_mfi"] = 'no_data'#заполняем пропущенные значения
    df["name_mfi"] = df["name_mfi"].str.lower()#ставим нижний регистр
    df_name_mfi = df['name_mfi'].value_counts().to_frame()#генерируем новый dataset из уникальных названий
    df_name_mfi['name_mfi_in'] = df_name_mfi.index #вытаскиваем уникальные названия в name_mfi_in
    translator = Translator()#активируем переводчик
    print('Подбираем язык и значения вероятности попадания')
    df_name_mfi["name_mfi_lang"] = df_name_mfi.name_mfi_in.progress_apply(translator.detect)#узнаем язык текста, вероятность правильности определения языка текста
    df_name_mfi["name_mfi_lang_atr1"] = df_name_mfi.name_mfi_lang.apply(getattr, args=('lang', ))  #язык текст
    df_name_mfi["name_mfi_lang_atr2"] = df_name_mfi.name_mfi_lang.apply(getattr, args=('confidence', ))#правильности определения языка текста
    df_name_mfi.to_csv('/content/drive/MyDrive/dataset_pochta/file_lang.csv', sep = ',',index=True, line_terminator = '\n') #сохраняем промежуточный результат

    c = df_name_mfi.loc[(df_name_mfi['name_mfi_lang_atr1'].isin(['zh-CN'])) | (df_name_mfi['name_mfi_lang_atr1'].isin( ['ru'])),['name_mfi_in']] # формируем массив = все что не английское
    print('переводим китайский и русский')
    c['name_mfi_translate'] = c['name_mfi_in'].progress_apply(translator.translate).apply(getattr, args=('text',)) # переводим
    df_name_mfi['name_mfi_translate'] = df_name_mfi.index.map(c.set_index(c.index)['name_mfi_translate']) #соединяем по name_mfi_in, нужно обработать чтобы переводил только китайский и русский
    #display(df_name_mfi)
    df_name_mfi.loc[(df_name_mfi['name_mfi_lang_atr1'].isin(['zh-CN'])) | (df_name_mfi['name_mfi_lang_atr1'].isin( ['ru'])),['name_mfi_in']] = df_name_mfi.loc[(df_name_mfi['name_mfi_lang_atr1'].isin(['zh-CN'])) | (df_name_mfi['name_mfi_lang_atr1'].isin( ['ru'])),['name_mfi_translate']]
    #print(df_name_mfi.loc[(df_name_mfi['name_mfi_lang_atr1'].isin(['zh-CN'])) | (df_name_mfi['name_mfi_lang_atr1'].isin( ['ru'])),['name_mfi_in']])
    nlp = spacy.load("en_core_web_sm") #загружаем ядро лемматизатора
    def remove_multiple_spaces(text): # оставим только буквы, они будут более точно отражать какие категории товаров пересылаются
        d = nlp(" ". join(re.sub(r'[^a-zA-Z ]', ' ', text).split())) #лемматизируем
        return ' '.join([token.lemma_ for token in d])#лемматизируем
    print('лемматизируем')
    df_name_mfi['name_mfi_translate'] = df_name_mfi['name_mfi_translate'].fillna(df_name_mfi['name_mfi_in'])
    df_name_mfi['name_mfi_in'] = df_name_mfi['name_mfi_translate']
    df_name_mfi["name_mfi_lemm"] = df_name_mfi.name_mfi_in.progress_apply(remove_multiple_spaces)#лемматизируем
    return df_name_mfi




In [ ]:
df_text = obrabotka_text(df_train)

Подбираем язык и значения вероятности попадания


  0%|          | 0/202808 [00:00<?, ?it/s]

переводим китайский и русский


  0%|          | 0/1565 [00:00<?, ?it/s]

лемматизируем


  0%|          | 0/202808 [00:00<?, ?it/s]

In [ ]:
df_text['index_id'] = df_text.index
df_text.to_csv('/content/drive/MyDrive/dataset_pochta/file_train_words1.csv', index_label=['index_id'])

In [ ]:
! [ ! -s "./test_dataset_test.csv" ] && curl -L -o test_dataset_test.csv "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1InOC4H4Qpu0eCvc0-DIgJ08LMzjnOya5"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  625M  100  625M    0     0  76.7M      0  0:00:08  0:00:08 --:--:-- 70.9M


In [ ]:
df_test = pd.read_csv("/content/test_dataset_test.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_text_2 = obrabotka_text(df_test)

Подбираем язык и значения вероятности попадания


  0%|          | 0/195628 [00:00<?, ?it/s]

переводим китайский и русский


  0%|          | 0/1473 [00:00<?, ?it/s]

лемматизируем


  0%|          | 0/195628 [00:00<?, ?it/s]

In [ ]:
df_text_2['index_id'] = df_text_2.index
df_text_2.to_csv('/content/drive/MyDrive/dataset_pochta/file_test_words.csv', index_label=['index_id'])

In [ ]:
df_text_1 = pd.read_csv("/content/drive/MyDrive/dataset_pochta/file_train_words.csv")

In [ ]:
print(df_text_2)

                                           name_mfi  \
no_data                                      544856   
phone case                                    50498   
mobile phone bags & cases                     32122   
screen protectors                             30161   
necklace                                      19329   
...                                             ...   
hairpin(32900879234),hairpin(32874318528)         1   
woman super high heels sandals                    1   
watchbands(dsb10011l58)                           1   
tg1711                                            1   
phone case(ys001-i11-a-)                          1   

                                                                         name_mfi_in  \
no_data                                                                      no_data   
phone case                                                                phone case   
mobile phone bags & cases                                  mobile phone bag

In [ ]:
df_text_2 = pd.read_csv("/content/drive/MyDrive/dataset_pochta/file_test_words.csv")

теперь напишем функцию по подсчету tf-idf и в последствии по алгоритму к-соседей kmeans раскидаем категории.

In [ ]:
gc.collect()
def clustering(df,df_t):
    corpus_train = df['name_mfi_lemm'].values.astype('U')
    count_tf_idf = TfidfVectorizer()
    tf_idf_train = count_tf_idf.fit_transform(corpus_train)
    kmeans = KMeans(n_clusters = 32)
    df['cluster'] = kmeans.fit_predict(tf_idf_train)


    corpus_test = df_t['name_mfi_lemm'].values.astype('U')
    tf_idf_test = count_tf_idf.transform(corpus_test)
    df_t['cluster'] = kmeans.predict(tf_idf_test)

    return(df, df_t)

In [ ]:
df_train_1, df_test_1= clustering(df_text_1, df_text_2)

In [ ]:
df_train_1.to_csv('/content/drive/MyDrive/dataset_pochta/file_train_words_clust.csv', index_label=['index_id'])
df_test_1.to_csv('/content/drive/MyDrive/dataset_pochta/file_test_words_clust.csv', index_label=['index_id'])

In [ ]:
display(df_train_1.groupby('cluster').count())
display(df_train_1[df_train_1['name_mfi_lang_atr1'] == 'ru'].head(50))
display(df_test_1.groupby('cluster').count())
display(df_test_1[df_test_1['name_mfi_lang_atr1'] == 'ru'].head(50))

,index_id,name_mfi,name_mfi_in,name_mfi_lang,name_mfi_lang_atr1,name_mfi_lang_atr2,name_mfi_translate,name_mfi_lemm,index_id.1
cluster,,,,,,,,,
0,2098,2098,2098,2098,2098,2098,2098,2098,2098
1,3074,3074,3074,3074,3074,3074,3074,3074,3074
2,105402,105402,105402,105402,105402,105402,105402,105189,105402
3,4641,4641,4641,4641,4641,4641,4641,4641,4641
4,3325,3325,3325,3325,3325,3325,3325,3325,3325
5,2940,2940,2940,2940,2940,2940,2940,2940,2940
6,3946,3946,3946,3946,3946,3946,3946,3946,3946
7,5072,5072,5072,5072,5072,5072,5072,5072,5072
8,4737,4737,4737,4737,4737,4737,4737,4737,4737


Int64Index([  4211,   4755,   4938,   4985,   5182,   5821,   5972,   6486,
              6636,   7532,
            ...
            199202, 199655, 199808, 199862, 201369, 201583, 201777, 201826,
            201887, 202517],
           dtype='int64', length=365)

,index_id,name_mfi,name_mfi_in,name_mfi_lang,name_mfi_lang_atr1,name_mfi_lang_atr2,name_mfi_translate,name_mfi_lemm,index_id.1
cluster,,,,,,,,,
0,2013,2013,2013,2013,2013,2013,2013,2013,2013
1,2955,2955,2955,2955,2955,2955,2955,2955,2955
2,101619,101619,101619,101619,101619,101619,101619,101422,101619
3,4518,4518,4518,4518,4518,4518,4518,4518,4518
4,3192,3192,3192,3192,3192,3192,3192,3192,3192
5,2822,2822,2822,2822,2822,2822,2822,2822,2822
6,3775,3775,3775,3775,3775,3775,3775,3775,3775
7,4893,4893,4893,4893,4893,4893,4893,4893,4893
8,4587,4587,4587,4587,4587,4587,4587,4587,4587


,index_id,name_mfi,name_mfi_in,name_mfi_lang,name_mfi_lang_atr1,name_mfi_lang_atr2,name_mfi_translate,name_mfi_lemm,index_id.1,cluster
3726,"серебряный орел g5,серебряный орел g5 беззвучн...",84,"silver eagle g5,silver eagle g5 silent wired m...","Detected(lang=ru, confidence=0.9707912)",ru,0.9707912,"silver eagle g5,silver eagle g5 silent wired m...",silver eagle g silver eagle g silent wire mous...,"серебряный орел g5,серебряный орел g5 беззвучн...",2
3968,"m4 умный браслет п,m4 умный браслет прямая про...",79,"m4 smart bracelet p,m4 smart bracelet factory ...","Detected(lang=ru, confidence=1)",ru,1,"m4 smart bracelet p,m4 smart bracelet factory ...",m smart bracelet p m smart bracelet factory di...,"m4 умный браслет п,m4 умный браслет прямая про...",2
4060,[мобильный телефон можно использовать] беспров...,76,"[mobile phone can be used] wireless,[mobile phone","Detected(lang=ru, confidence=0.96839774)",ru,0.96839774,"[mobile phone can be used] wireless,[mobile phone",mobile phone can be use wireless mobile phone,[мобильный телефон можно использовать] беспров...,14
4156,"bluetooth-гарнитура e6s, влагозащищенные мини-...",74,"e6s bluetooth headset, waterproof mini earphon...","Detected(lang=ru, confidence=0.7136391)",ru,0.7136391,"e6s bluetooth headset, waterproof mini earphon...",e s bluetooth headset waterproof mini earphone...,"bluetooth-гарнитура e6s, влагозащищенные мини-...",2
4299,"беспроводная гар,беспроводная гарнитура tws с ...",71,"wireless gar,tws wireless headset with bluetoo...","Detected(lang=ru, confidence=0.74815524)",ru,0.74815524,"wireless gar,tws wireless headset with bluetoo...",wireless gar tws wireless headset with bluetoo...,"беспроводная гар,беспроводная гарнитура tws с ...",2
4719,m4 умный браслет прямая продажа с фабрики цвет...,64,"m4 smart bracelet factory direct sale color,m4...","Detected(lang=ru, confidence=1)",ru,1,"m4 smart bracelet factory direct sale color,m4...",m smart bracelet factory direct sale color m s...,m4 умный браслет прямая продажа с фабрики цвет...,2
6249,"горячая продажа б,горячая продажа беспроводная...",47,"hot sale b,hot sale wireless bluetooth headset t","Detected(lang=ru, confidence=0.988031)",ru,0.988031,"hot sale b,hot sale wireless bluetooth headset t",hot sale b hot sale wireless bluetooth headset t,"горячая продажа б,горячая продажа беспроводная...",2
6678,"прямая продажа, 14 ,прямая продажа, 14 цветов,...",44,"direct sale, 14 , direct sale, 14 colors, glit...","Detected(lang=ru, confidence=1)",ru,1,"direct sale, 14 , direct sale, 14 colors, glit...",direct sale direct sale color glitter eye shadow,"прямая продажа, 14 ,прямая продажа, 14 цветов,...",2
6721,"bluetooth-гарнитура e6s,bluetooth-гарнитура e6...",43,"bluetooth headset e6s,bluetooth headset e6s, w...","Detected(lang=ru, confidence=0.7136391)",ru,0.7136391,"bluetooth headset e6s,bluetooth headset e6s, w...",bluetooth headset e s bluetooth headset e s wa...,"bluetooth-гарнитура e6s,bluetooth-гарнитура e6...",2
7255,серебряный орел g5 беззвучная проводная мышь с...,40,"silver eagle g5 silent wired mouse with shes,s...","Detected(lang=ru, confidence=0.9707912)",ru,0.9707912,"silver eagle g5 silent wired mouse with shes,s...",silver eagle g silent wire mouse with she s si...,серебряный орел g5 беззвучная проводная мышь с...,2


In [ ]:
import zipfile

Вывод: мы подготовили текстовые данные, разбили товары по категориям, выявили языки и правильность перевода которые можно также использовать ка категориальные и количественные признаки (правильность перевода). при этом сохранили индексы для распределения в финальном датасете.

In [ ]:
files = '/content/'
archive = "archive.zip"


with zipfile.ZipFile(archive, "w") as zf:

    zf.write('/content')

In [ ]:
!tar

tar: You must specify one of the '-Acdtrux', '--delete' or '--test-label' options
Try 'tar --help' or 'tar --usage' for more information.


In [ ]:
!tar -cvjf files.tar.bz2 /content/*.whl

tar: Removing leading `/' from member names
/content/absl_py-1.4.0-py3-none-any.whl
tar: Removing leading `/' from hard link targets
/content/accelerate-0.17.1-py3-none-any.whl
/content/aiohttp-3.8.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/content/aiosignal-1.3.1-py3-none-any.whl
/content/alembic-1.10.2-py3-none-any.whl
/content/anyio-3.6.2-py3-none-any.whl
/content/APScheduler-3.10.1-py3-none-any.whl
/content/arrow-1.2.3-py3-none-any.whl
/content/astunparse-1.6.3-py2.py3-none-any.whl
/content/async_timeout-4.0.2-py3-none-any.whl
/content/attrs-22.2.0-py3-none-any.whl
/content/Babel-2.12.1-py3-none-any.whl
/content/backoff-1.11.1-py2.py3-none-any.whl
/content/beautifulsoup4-4.11.2-py3-none-any.whl
/content/binaryornot-0.4.4-py2.py3-none-any.whl
/content/black-22.3.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/content/boto3-1.26.90-py3-none-any.whl
/content/botocore-1.29.90-py3-none-any.whl
/content/cachetools-5.3.0-py3-none-any.whl
/content/certifi-2022.